<a href="https://colab.research.google.com/github/morizk/Sift_Feature_Detector_descriptor_matching/blob/main/Sift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import cv2 as cv2
import scipy.stats as st


def harris_corner_detector(img_gray,points,Gaussian_size=15,Gaussian_sigma=12,Window_size=10):
  points_shape=np.shape(points)
  points_holder=np.zeros((points_shape[0],2))
  # the window of the Harris Matrix
  pad_size=int((Window_size-1)/2)

  img_gray=np.array(img_gray)

  #appling a Gussian filter 
  img_blur = cv2.GaussianBlur(img_gray,(Gaussian_size,Gaussian_size),Gaussian_sigma)

  #calculting I_x and I_y using sobel matrix

  Sobel_X=np.array([[1,2,1],[0,0,0],[-1,-2,-1]])
  Sobel_Y=np.array([[1,0,-1],[2,0,-2],[1,0,-1]])
  I_x=cv2.filter2D(img_blur,-1,Sobel_X)
  I_y=cv2.filter2D(img_blur,-1,Sobel_Y)

  #normilzing the pixel value (make it eaiser to understand R)
  I_x=I_x/np.max(I_x)
  I_y=I_y/np.max(I_y)
  #pading the Is
  I_x_y=np.pad(I_x*I_y, ((pad_size, pad_size), (pad_size, pad_size)), 'constant', constant_values=0)
  I_xx=np.pad(pow(I_x,2), ((pad_size, pad_size), (pad_size, pad_size)), 'constant', constant_values=0)
  I_yy=np.pad(pow(I_y,2), ((pad_size, pad_size), (pad_size, pad_size)), 'constant', constant_values=0)
  img_shape=np.shape(img_gray)
  ind=0
  for point in points:
      h=int(point[0])
      w=int(point[1])
      #calculing the Harris matrix over a sliding window
      d_x=np.sum(I_xx[h:h+Window_size,w:w+Window_size])
      d_y=np.sum(I_yy[h:h+Window_size,w:w+Window_size])
      d_xy=np.sum(I_x_y[h:h+Window_size,w:w+Window_size])


      det=d_x*d_y-pow(d_xy,2)
      trace=d_x+d_y
      if det==0:
        det=.00000001

      y=(pow(trace,2)/det)
      R=det-.05*pow(trace,2)
      # R was add because so point in a flat area were selected
      if  y<12.5 and R>1:
        points_holder[ind]=[h,w]
        ind +=1



  out=points_holder[0:ind,:]

  return  out.astype(int)

def get_interest_points(image,width_f):

    scale=[]
    #as the feature width is 16; 4 octave has been proven better by trial and error. more octave can be used with a bigger descriptor
    number_of_octave=4
    number_of_blurs=5
    #the constant that will be multiplied by sigma
    k=pow(2,.5)
    print("creating the scale and blur space...")
    for i in range(number_of_octave):
        #the base sigma will increase with every octave
        sigma=1.6*(i+1)
        #first octave will be douple thw image
        scale_change=1/(pow(2,(i-1)))
        width = int(image.shape[1] * scale_change)
        height = int(image.shape[0] * scale_change)
        dim = (width, height)
        base_image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        octave=np.zeros((height,width,number_of_blurs))
        for ii in range(number_of_blurs):
            octave[:,:,ii]=cv2.GaussianBlur(base_image,(0,0),sigmaX=sigma)
            sigma=sigma*k
        scale.append(octave)
    print("done")
    print("    ")
    print("calculating DOG...")
    DOG=[]
    for i in range(number_of_octave):
       octave=scale[i]
       DOG_octave_size=list(np.shape(octave))
       DOG_octave_size[2]=number_of_blurs-1
       octave_holder=np.zeros(DOG_octave_size)
       for ii in range(number_of_blurs-1):
            octave_holder[:,:,ii]=octave[:,:,ii]-octave[:,:,ii+1]
       DOG.append(octave_holder)
    print("done")
    print("    ")
    print("checking the local max and min in scale space...")
    can_point=[]

    for i in range(number_of_octave):
        octave_holder = DOG[i]
        octave_shape = np.array(np.shape(octave_holder))
        octave_shape=octave_shape-2
        octave = np.zeros(octave_shape)
        octave_holder = np.where(np.abs(octave_holder) < .03, 0, octave_holder)
        for h in range(octave_shape[0]):
            for w in range(octave_shape[1]):
                for d in range(octave_shape[2]):
                    cube=octave_holder[h:h+3,w:w+3,d:d+3]
                    if np.argmin(cube) == 13 or np.argmax(cube) == 13:
                        octave[h,w,d]=cube[1,1,1]

        can_point.append(octave)
    print("done")
    print("    ")
    print("gatting the points from all scales...")
    points=[]
    for i in range(number_of_octave):
        octave_holder=can_point[i]
        x_scale =image.shape[0]/np.shape(octave_holder)[0]
        y_scale =image.shape[1]/np.shape(octave_holder)[1]
        points_holder=np.nonzero(octave_holder)
        points_holder=np.rot90(np.array([points_holder[0],points_holder[1]]))
        points.append(points_holder*[x_scale,y_scale])

    points=np.concatenate( points, axis=0 )
    print("done")
    print("    ")
    print("removing edge and weak corner...")
    good=harris_corner_detector(image,points)
    print("done")
    print("   ")
    good=np.unique(good, axis=0)
    xs = good[:,1]
    ys = good[:,0]
    print(np.shape(xs)[0]," points were found")
    return xs, ys
# a gussian to wight the magnitude of the discrebtor
def gkern(kernlen, nsig):

    x = np.linspace(-nsig, nsig, kernlen+1)
    kern1d = np.diff(st.norm.cdf(x))
    kern2d = np.outer(kern1d, kern1d)
    return kern2d/kern2d.sum()

#calculating the main angles of the point
def cal_shift(window):
  bin=36
  shift=[]
  shape_wind=np.shape(window)[0]
  dis=np.zeros((shape_wind-2,shape_wind-2,2))
  for h in range(1,shape_wind-1):
    for w in range(1,shape_wind-1):
      m=pow(pow(window[h+1,w]-window[h-1,w],2)+pow(window[h,w+1]-window[h,w-1],2),.5)
      base=(window[h+1,w]-window[h-1,w])
      if base ==0 :
        base=.0001
      o=angle_correction(np.arctan((window[h,w+1]-window[h,w-1])/base)*(180/np.pi))
      dis[h-1,w-1,0]=m
      dis[h-1,w-1,1]=o
  histogram=histogram_calculator(dis,bin)
  orint=np.flipud(np.argsort(histogram, axis=0))
  for i in range(np.shape(orint)[0]):
    if histogram[orint[i]] > .8 * histogram[orint[0]]:
      shift.append(orint[i]*(360/bin))
  return np.array(shift)

#creating histogram
def histogram_calculator(dis,bin):
  histogram=np.zeros((bin,1))
  for h in range(np.shape(dis)[0]):
    for w in range(np.shape(dis)[1]):
      o=dis[h,w,1]
      m=dis[h,w,0]
      ind=int(o // (360/bin))
      if ind==bin:
        ind=bin-1
      #d=0
      #the d used to calculate trilinear interpolation
      d=np.abs(((360/bin)/2)+(ind*(360/bin))-o)/(360/bin)
      histogram[ind,0]=histogram[ind,0]+m*(1-d)
  return np.array(histogram)
#to deal with angle less than 0 or more than 360
def angle_correction(angle):
  if angle>360:
    n= angle //360
    angle=angle-n*360
  elif angle<0:
    n=np.abs(angle//360)
    angle=angle+n*360
  return int(angle)



#calculate the magnitude and the angles of  discrebtor
def id_calculator( window , shift ):
  window_len=np.shape(window)[0]
  bin=8
  size_histogram=pow(int((window_len-2)//4),2)*bin
  dis=np.zeros((window_len-2,window_len-2,2))
  histogram=np.zeros((size_histogram,1))
  his_ind=0
  for h in range(1,window_len-1):
    for w in range(1,window_len-1):
      m=pow(pow(window[h+1,w]-window[h-1,w],2)+pow(window[h,w+1]-window[h,w-1],2),.5)
      base=(window[h+1,w]-window[h-1,w])
      if base ==0 :
        base=.00001
      o=angle_correction(angle_correction(np.arctan((window[h,w+1]-window[h,w-1])/base)*(180/np.pi))-shift)
      dis[h-1,w-1,0]=m
      dis[h-1,w-1,1]=o
  #decrease the magnitude of far away pixel
  gaussian_weights=gkern((np.shape(window)[0]-2), 1.5*(np.shape(window)[0]-2)/16)
  dis[:,:,0]=dis[:,:,0]*gaussian_weights

  for h in range(0,np.shape(dis)[0],4):
    for w in range(0,np.shape(dis)[0],4):
      histogram[his_ind:his_ind+bin,[0]]=histogram_calculator(dis[h:h+4,w:w+4,:],bin)
      his_ind=bin+his_ind
  return np.rot90(histogram)




def get_features(image, x, y, feature_width):
  holder=x
  x=y
  y=holder
  image=image*255
  assert np.shape(x) == np.shape(y)
  assert feature_width % 4 == 0
  pad_up = int(feature_width/2)
  pad_down = int((feature_width/2)+1)
  image=np.pad(image, ((pad_up,pad_down), (pad_up,pad_down)), 'edge')
  len_feature=int(pow(feature_width/4,2))*8
  features=[]
  for i in range(np.shape(x)[0]):
    window=image[x[i]+pad_up-3:x[i]+pad_up+4,y[i]+pad_up-3:y[i]+pad_up+4]
    shift=cal_shift(window)
    window_disc=image[x[i]:x[i]+feature_width+2,y[i]:y[i]+feature_width+2]
    feature_of_point=np.zeros((np.shape(shift)[0],len_feature))
    #calculate all possible discrebtor by the main angles of the point
    for ii in range(np.shape(shift)[0]):
      feature_o=id_calculator(window_disc,shift[ii])
      feature_o= feature_o / np.linalg.norm(feature_o)
      feature_o=np.where(feature_o>.2,.2,feature_o)
      feature_of_point[ii,:]= feature_o / np.linalg.norm(feature_o)
    features.append(feature_of_point)
  return features

def distance_cal(f_1,f_2):
  distance_m=np.zeros((np.shape(f_1)[0],np.shape(f_2)[0]))
  for h in range(np.shape(f_1)[0]):
    for w in range(np.shape(f_2)[0]):
      distance_m[h,w]=np.linalg.norm(f_1[h,:]-f_2[w,:])

  return np.min(distance_m)
def match_features(im1_features, im2_features):
    match_con=[]
    for i in  range(np.shape(im1_features)[0]):
      distance=np.zeros((np.shape(im2_features)[0],1))
      for ii in range(np.shape(im2_features)[0]):
        distance[ii,0]=distance_cal(im1_features[i],im2_features[ii])
      arranged=np.sort(distance, axis=0)
      indx=np.argsort(distance, axis=0)
      con=int(np.exp(-.2*arranged[0,0])*100)
      if arranged[0,0] < arranged[1,0]-.05*arranged[1,0] and con >50 :
        match_con.append([i,indx[0,0],con])
    match_con=np.array(match_con)
    if np.shape(match_con)[0]==0:
      print("no match !!!")
      matches=[]
      confidences=[]
    else:
      matches = match_con[:,0:2]
      confidences = match_con[:,2]
    return matches, confidences
student.py
Displaying student.py.